### The Agenda of this notebook is just to deep dive into the data, perform some EDA to learn some patterns and then these points will help us in Feature Engineering

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [40]:
url = 'https://drive.google.com/file/d/1GRRfOnM0cwB9cRlyUVZHw0com64VFz9o/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, encoding='iso-8859-1', on_bad_lines='skip')

In [41]:
df = df[df['Retailer Name'].isin(['LONDON DRUGS', 'JEAN-COUTU'])]

In [42]:
df.head()

,Retailer Code,Retailer Name,Door Code,Door Name,Door Zip Code,Year,Week,UPC,SKU Category,SKU Descreption,POS Units,POS Amount
144,LON,LONDON DRUGS,216861,LONDON DRUGS #60,T1B 1E2,2022,152022,8435137727100,NaN,PRADA CANDY EDP V30ML V0,1.0,85.0
145,LON,LONDON DRUGS,239308,LONDON DRUGS #86,S4W 0B7,2022,152022,3360372058861,NaN,GA ADGH EDT SP 50ML SUB,1.0,95.0
146,LON,LONDON DRUGS,239308,LONDON DRUGS #86,S4W 0B7,2022,152022,3365440375055,NaN,YSL NUIT EDT SPRAY 60ML SUB,1.0,96.0
147,LON,LONDON DRUGS,221758,LONDON DRUGS #65,S7K 5S6,2022,152022,3360372058861,NaN,GA ADGH EDT SP 50ML SUB,1.0,95.0
148,LON,LONDON DRUGS,213529,LONDON DRUGS #09,V3R 1B9,2022,152022,3605970326494,NaN,LANC RML CR NUIT J75ML SUB,1.0,148.0


### Model to find the products allocation for each Retailer

### Product Allocation for London Drugs

In [43]:
model_df_ld = df.loc[df['Retailer Name'] == 'LONDON DRUGS'][['Retailer Code', 'UPC', 'POS Units']]
model_df_ld.head()

,Retailer Code,UPC,POS Units
144,LON,8435137727100,1.0
145,LON,3360372058861,1.0
146,LON,3365440375055,1.0
147,LON,3360372058861,1.0
148,LON,3605970326494,1.0


In [44]:
unique_upcs = model_df_ld['UPC'].unique()
unique_upcs.shape

(801,)

In [45]:
model_df_ld_allocated = model_df_ld[model_df_ld['POS Units']>0]
model_df_ld_returned = model_df_ld[model_df_ld['POS Units']<0]

### Prepare the dataset where find the appropriate number of items against each upc i.e., UPC = items allocated - items returned for each retailer i.e UPC 123 is being allocated 10 times to London Drugs and 13 times to Jean-Coutu and returned 7 times from London drugs and 5 times from Jean-Coutu so the desired data format would be:

London Drugs = 10-7 = 3
Jean Coutu = 13-5 = 8

In [46]:
len(unique_upcs)

801

In [47]:
ld_adjusted_products = {}
for upc in unique_upcs:
    #get the sum of all the products that were allocated for each upc
    allocated_units = sum(model_df_ld_allocated[model_df_ld_allocated['UPC'] == upc]['POS Units'])
    returned_units = sum(model_df_ld_returned[model_df_ld_returned['UPC'] == upc]['POS Units'])
    ld_adjusted_products[upc] = allocated_units - returned_units

In [48]:
len(list(ld_adjusted_products.keys()))

801

### Product Allocation for JEAN-COUTU

In [49]:
model_df_jc = df.loc[df['Retailer Name'] == 'JEAN-COUTU'][['Retailer Code', 'UPC', 'POS Units']]
model_df_jc.head()

,Retailer Code,UPC,POS Units
2473,PJC,3614270129681,1.0
2474,PJC,3614270129681,1.0
2475,PJC,3614270129681,1.0
2476,PJC,3614270129681,1.0
2477,PJC,3614270129681,1.0


In [50]:
unique_upcs = model_df_jc['UPC'].unique()
unique_upcs.shape

(1002,)

In [51]:
model_df_jc_allocated = model_df_jc[model_df_jc['POS Units']>0]
model_df_jc_returned = model_df_jc[model_df_jc['POS Units']<0]

### Prepare the dataset where find the appropriate number of items against each upc i.e., UPC = items allocated - items returned for each retailer i.e UPC 123 is being allocated 10 times to London Drugs and 13 times to Jean-Coutu and returned 7 times from London drugs and 5 times from Jean-Coutu so the desired data format would be:

London Drugs = 10-7 = 3
Jean Coutu = 13-5 = 8

In [52]:
len(unique_upcs)

1002

In [53]:
jc_adjusted_products = {}
for upc in unique_upcs:
    #get the sum of all the products that were allocated for each upc
    allocated_units = sum(model_df_jc_allocated[model_df_jc_allocated['UPC'] == upc]['POS Units'])
    returned_units = sum(model_df_jc_returned[model_df_jc_returned['UPC'] == upc]['POS Units'])
    jc_adjusted_products[upc] = allocated_units - returned_units

In [54]:
len(list(jc_adjusted_products.keys()))

1002

In [55]:
jc_upcs = len(jc_adjusted_products.keys())
ld_upcs = len(ld_adjusted_products.keys())

In [56]:
retailers = []
retailers.extend(['London Drugs' for x in range(ld_upcs)])
retailers.extend(['Jean Coutu' for x in range(jc_upcs)])
len(retailers)

1803

In [57]:
upcs_for_each_retailer = []
upcs_for_each_retailer.extend([x[0] for x in jc_adjusted_products.items()])
upcs_for_each_retailer.extend([x[0] for x in ld_adjusted_products.items()])
len(upcs_for_each_retailer)

1803

In [58]:
adjusted_upc_amount_for_each_retailer = []
adjusted_upc_amount_for_each_retailer.extend([x[1] for x in jc_adjusted_products.items()])
adjusted_upc_amount_for_each_retailer.extend([x[1] for x in ld_adjusted_products.items()])
len(adjusted_upc_amount_for_each_retailer)

1803

In [59]:
product_allocation_for_each_retailer = pd.DataFrame({'Retailer':retailers, 
                                                   'UPC': upcs_for_each_retailer,
                                                   'Unit': adjusted_upc_amount_for_each_retailer})

In [60]:
product_allocation_for_each_retailer.tail()

,Retailer,UPC,Unit
1798,Jean Coutu,3614272889491,15.0
1799,Jean Coutu,3614270256332,2.0
1800,Jean Coutu,772191735237,3.0
1801,Jean Coutu,3614271412591,2.0
1802,Jean Coutu,3605970761134,1.0


In [61]:
retail_dict = {'London Drugs':0, 'Jean Coutu': 1}

In [62]:
product_allocation_for_each_retailer['retailer'] = product_allocation_for_each_retailer['Retailer'].map(retail_dict)

In [63]:
product_allocation_for_each_retailer.drop(['Retailer'], axis=1, inplace=True)

In [64]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [65]:
X = product_allocation_for_each_retailer.drop(['Unit'], axis=1)
y = product_allocation_for_each_retailer['Unit']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [67]:
model = RandomForestRegressor()

In [68]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [69]:
from joblib import dump, load

In [70]:
dump(model, 'UseCase1.h5')

['UseCase1.h5']

In [71]:
preds = model.predict(X_test)

In [72]:
from sklearn.metrics import r2_score
r2_score(y_test, preds)

-0.006390587639772294

### UPCS with 0 allocation

In [73]:
for d in jc_adjusted_products.items():
    if d[1] == 0.0:
        print(d[0])